Datasets: 

**Including Text**

https://www.kaggle.com/datasets/michaelrussell4/10000-books-and-their-genres-standardized

**Including Publication Date**

https://www.kaggle.com/datasets/saurabhbagchi/books-dataset

In this notebook, we aim to merge multiple datasets into one df in order to do further analysis.

We aim to get the 'text' column of what we will be labelling as `text_df` , append a date via the `date_df`.

# Imports

In [1]:
import pandas as pd

# Load Dataset

In [2]:
# Read the CSV file into a DataFrame
text_df = pd.read_csv('datasets/books_and_genres.csv')

In [3]:
date_df = pd.read_csv('datasets/books.csv', encoding='ISO-8859-1', on_bad_lines='skip', sep=';', dtype={'Year-Of-Publication': 'str'})

In [20]:
print(text_df.isnull().sum(), '\n\n', date_df.isnull().sum())

Unnamed: 0      0
title           0
text          262
genres          0
dtype: int64 

 ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64


# EDA

## Basic Inspection of data

In [5]:
text_df.head()

,Unnamed: 0,title,text,genres
0,0,apocolocyntosis,"Produced by Ted Garvin, Ben Courtney and PG Di...","{'21st-century', 'history', 'roman', 'classics..."
1,1,the house on the borderland,"Produced by Suzanne Shell, Sjaani and PG Distr...","{'horror', 'mystery', 'classics', 'science-fic..."
2,2,the warriors,"Produced by Charles Aldarondo, Charlie Kirschn...","{'literary-fiction', 'history', 'biography', '..."
3,3,a voyage to the moon,"Produced by Christine De Ryck, Stig M. Valstad...","{'20th-century', 'science-fiction', 'speculati..."
4,4,la fiammetta,"Produced by Ted Garvin, Dave Morgan and PG Dis...","{'literary-fiction', 'history', 'feminism', 'c..."


In [6]:
date_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [7]:
df.shape

NameError: name 'df' is not defined

In [ ]:
df.info()

In [ ]:
df = df.dropna(subset=['text'])

In [ ]:
df.info()

In [ ]:
df.shape